Lecture 06.

In [ ]:
from torch import tensor
from torch import nn
from torch import sigmoid
import torch.optim as optim

x_data = tensor([[1.0],[2.0],[3.0],[4.0]])
y_data = tensor([[0.],[0.],[1.0],[1.0]])

class Model(nn.Module):
  # 생성자와 forward를 오버라이딩한다 (= 추가한다)
  def __init__(self):
    super(Model, self).__init__()
    self.linear = nn.Linear(1,1) # 1개의 노드로 입력을 받아 1개의 출력을 낸다, 레이어는 하나

  def forward(self, x): # 실제 모델이 동작하는 부분
    y_pred = sigmoid(self.linear(x))
    return y_pred

model = Model()

criterion = nn.BCELoss(reduction='mean') # 1/n을 곱해준다는 것. 설정 안하면 그냥 sum만 함
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1000):
  y_pred = model(x_data)

  loss = criterion(y_pred, y_data)
  print(f'Epoch {epoch+1}/1000 | Loss: {loss.item(): .4f}')

  optimizer.zero_grad() # gradient 0으로 초기화
  loss.backward() # 각 파라미터에 대한 gradient를 구함
  optimizer.step() # 가중치 업데이트

# After training
print(f'\nLet\'s predict the hours need to score above 50%\n{"=" * 50}')
hour_var = model(tensor([[1.0]]))
print(f'Prediction after 1 hour of training: {hour_var.item():.4f} | Above 50%: {hour_var.item() > 0.5}')
hour_var = model(tensor([[7.0]]))
print(f'Prediction after 7 hours of training: {hour_var.item():.4f} | Above 50%: { hour_var.item() > 0.5}')

Lecture 07.

In [ ]:
from torch import nn, optim, from_numpy
import numpy as np
from google.colab import drive

drive.mount('/content/drive')
xy = np.loadtxt('/content/drive/My Drive/머신러닝 수업 실습/diabetes.csv.gz', delimiter=',',dtype=np.float32)
x_data = from_numpy(xy[:, 0:-1])
y_data = from_numpy(xy[:, [-1]])
print(f"XW's shape: {x_data.shape} | YW's shape: {y_data.shape}")

class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.l1 = nn.Linear(8,6)
    self.l2 = nn.Linear(6,4) # 이전 output 노드와 앞의 input 노드의 수가 같아야 함
    self.l3 = nn.Linear(4,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out1 = self.sigmoid(self.l1(x))
    out2 = self.sigmoid(self.l2(out1))
    y_pred = self.sigmoid(self.l3(out2))
    return y_pred

model = Model()

criterion = nn.BCELoss(size_average = True)
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(1000):
  y_pred = model(x_data)
  loss = criterion(y_pred, y_data)
  print(f'Epoch: {epoch+1}/1000 | Loss: {loss.item(): .4f}')

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Exercise 7-1 (More than 10 layers, another activation function, minimizing loss)

In [ ]:
from torch import nn, optim, from_numpy
import numpy as np
from google.colab import drive
import torch.nn.functional as F

drive.mount('/content/drive')

xy = np.loadtxt('/content/drive/My Drive/머신러닝 수업 실습/diabetes.csv.gz', delimiter=',',dtype=np.float32)

x_data = from_numpy(xy[:, 0:-1])
y_data = from_numpy(xy[:, [-1]])

print(f"XW's shape: {x_data.shape} | YW's shape: {y_data.shape}")

In [ ]:
# 레이어를 10층 이상으로 늘림
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.l1 = nn.Linear(8,7)
    self.l2 = nn.Linear(7,6) # 이전 output 노드와 앞의 input 노드의 수가 같아야 함
    self.l3 = nn.Linear(6,6)
    self.l4 = nn.Linear(6,6)
    self.l5 = nn.Linear(6,5)
    self.l6 = nn.Linear(5,5)
    self.l7 = nn.Linear(5,5)
    self.l8 = nn.Linear(5,4)
    self.l9 = nn.Linear(4,4)
    self.l10 = nn.Linear(4,4)
    self.l11 = nn.Linear(4,1)

# 시그모이드 대신 relu 활성화 함수를 사용함, 마지막엔 분류를 위해 시그모이드
  def forward(self, x):
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    x = F.relu(self.l4(x))
    x = F.relu(self.l5(x))
    x = F.relu(self.l6(x))
    x = F.relu(self.l7(x))
    x = F.relu(self.l8(x))
    x = F.relu(self.l9(x))
    x = F.relu(self.l10(x))
    y_pred =  F.sigmoid(self.l11(x))
    return y_pred

model = Model()

# 옵티마이저를 SGD 대신 Adam을 사용했더니 loss가 많이 줄음
criterion = nn.BCELoss(size_average = True)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1000):
  y_pred = model(x_data)
  loss = criterion(y_pred, y_data)
  print(f'Epoch: {epoch+1}/1000 | Loss: {loss.item(): .4f}')

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Exercise 7-1 (Try other datasets, iris)

In [ ]:
from torch import nn, optim, from_numpy
import pandas as pd
import numpy as np
from google.colab import drive
import torch.nn.functional as F

drive.mount('/content/drive')

xy = pd.read_csv('/content/drive/My Drive/머신러닝 수업 실습/Iris.csv', delimiter=',',dtype=np.float32)
xy = np.array(xy)

x_data = from_numpy(xy[:, 0:-1])
y_data = from_numpy(xy[:, [-1]])

print(f"XW's shape: {x_data.shape} | YW's shape: {y_data.shape}")

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.l1 = nn.Linear(4,4)
    self.l2 = nn.Linear(4,4)
    self.l3 = nn.Linear(4,4)
    self.l4 = nn.Linear(4,3)

  def forward(self, x):
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = F.relu(self.l3(x))
    y_pred =  F.relu(self.l4(x))
    return y_pred

model = Model()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

y_pred = model(x_data)

for epoch in range(1000):
  y_pred = model(x_data)
  loss = criterion(y_pred, y_data)
  print(f'Epoch: {epoch+1}/1000 | Loss: {loss.item(): .4f}')

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()